In [47]:
#import Libraries
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

In [48]:
# get wiki datasets
pathForWikiDataset = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
wikiSentences = WikiCorpus(pathForWikiDataset).get_texts()
print(wikiSentences)

<generator object WikiCorpus.get_texts at 0x7f3e658b5fc0>


In [49]:
# get Reuters DataSet
nltk.download('punkt')
nltk.download('reuters')
reutersSentences  = reuters.sents()
print(reutersSentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


## Create Sentence Model and Calculate Probability

In [59]:
def calculateProbabilities(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count

In [51]:
def calculateSigleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word]/=wordCount


In [52]:
def convertToLower(pa):
  if type(pa)==str:
    return pa.lower()
  return pa

In [53]:
sentenceModel4 = defaultdict(lambda: set())
sentenceModel5 = defaultdict(lambda: set())

def calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    #print(sentence)
    for word in sentence:
      wordCount+=1
      sentenceModel1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      # print(previousWord1)
      # print(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1]+=1
      sentenceModel3[nextWord][previousWord2]+=1
      sentenceModel4[previousWord1].add(nextWord)
      sentenceModel5[previousWord2].add(nextWord)

  return wordCount

In [54]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))


In [55]:
wikiWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,wikiSentences)
print(wikiWordCount)

452944


In [57]:
reutersWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,reutersSentences)
print(reutersWordCount)

1720917


In [60]:
calculateProbabilities(sentenceModel2)

In [61]:
calculateProbabilities(sentenceModel3)

In [62]:
total_word = wikiWordCount + reutersWordCount
calculateSigleWordProbability(sentenceModel1,total_word)

## Naive Bias

In [65]:
maxProbabilityWords = []
def makeWordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in sentenceModel4[previousWord1] & sentenceModel5[previousWord2]:
    naiveBiasTrigramWeight = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2]
    maxProbabilityWords.append((nextWord,naiveBiasTrigramWeight))

In [66]:
makeWordSuggestionByTrigram('my','name')
maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*maxProbabilityWords[:10])

('is', 6.312802951590447e-08) ('to', 2.3040750252019746e-09) (',', 1.118876979391946e-09) ('for', 5.523960654537554e-10) ('in', 4.657943430020126e-10) ('and', 3.770658245232208e-10) ('will', 3.324564975158133e-10) ('that', 2.4438966373356113e-10) ('would', 2.3203292613177524e-10) ('at', 2.1806795838226355e-10)


In [71]:
#Input only two words 
while(True):
    text = input("Enter your line: ")
    if text == "stop":
        print("Ending The Program.....")
        break
    
    else:
        try:
            maxProbabilityWords = []
            text = text.split(" ")
            makeWordSuggestionByTrigram(text[0],text[1])
            maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
            print(*maxProbabilityWords[:10])
            
        except:
            continue

Enter your line: i hate
('it', 3.904520837343442e-09) ('to', 3.560843220766688e-10) ('and', 2.7422969056234244e-11)
Enter your line: what is 
('now', 1.1189113303911986e-06) ('happening', 9.813568270157722e-07) ('called', 6.245913076367109e-07) ('really', 5.125836998252024e-07) ('termed', 4.870704839967988e-07) ('aesthetically', 3.6800881013091455e-07) ('thought', 3.6000861860632945e-07) ('going', 3.3605255655523216e-07) ('viewed', 3.0667400844242877e-07) ('required', 2.784303957544125e-07)
Enter your line: my name
('is', 6.312802951590447e-08) ('to', 2.3040750252019746e-09) (',', 1.118876979391946e-09) ('for', 5.523960654537554e-10) ('in', 4.657943430020126e-10) ('and', 3.770658245232208e-10) ('will', 3.324564975158133e-10) ('that', 2.4438966373356113e-10) ('would', 2.3203292613177524e-10) ('at', 2.1806795838226355e-10)
Enter your line: you are
('not', 2.3005177223427501e-07) ('re', 1.3925936869318837e-07) ('going', 1.2812003718668224e-07) ('still', 1.0018504367421032e-07) ('certainly